In [1]:
import shapefile
import shapely

In [3]:
polygons_sf = shapefile.Reader("/Users/damoncrockett/Desktop/FISP/SD/SD_TILE_SHAPE/SD_TILE_SHAPE.shp")

In [5]:
records = polygons_sf.records()

In [6]:
polygon_shapes = polygons_sf.shapes()
polygon_points = [q.points for q in polygon_shapes]
from shapely.geometry import Polygon, Point
polygons = [Polygon(q) for q in polygon_points]

In [7]:
len(polygons)

54

In [8]:
len(records)

54

In [13]:
polygons_sf.fields

[('DeletionFlag', 'C', 1, 0),
 ['tileName', 'C', 254, 0],
 ['fileName', 'C', 254, 0],
 ['prodDesc', 'C', 254, 0],
 ['volNum', 'C', 254, 0]]

In [16]:
from collections import Counter

In [21]:
Counter([item[2] for item in records])

Counter({'SD': 54})

In [33]:
polygon_points[4]

[[-117.17532811498636, 33.05644813475931],
 [-117.17532811498636, 33.12950950061074],
 [-117.13152478637844, 33.12950950061074],
 [-117.13152478637844, 33.05644813475931],
 [-117.17532811498636, 33.05644813475931]]

#### compare with SANDAG polygon

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt

In [10]:
from xml.etree import ElementTree
import keytree
from shapely.geometry import Polygon, Point, shape

doc = open("/Users/damoncrockett/vosd.org/215-opendsd/Final.kml").read()
tree = ElementTree.fromstring(doc)
# kml namespace
kmlns = tree.tag.split('}')[0][1:]
# find all placemarks
placemks = tree.findall(".//{%s}Placemark" % kmlns)
# filter out those without polygon elements
placemks_with_polygons = []

for p in placemks:
    if p.findall(".//{%s}Polygon" % kmlns):
        placemks_with_polygons.append(p)
        
# func: extract kml LinearRings, convert to shapely LinearRings, make dict with cpa names

def coords_names(placemks):
  coords_names_dict = {}
  for placemk in placemks:
    name = placemk.getchildren()[0].text
    coord_text = placemk.findtext(".//{%s}coordinates" % kmlns)
    coords = []
    for elems in coord_text.split():
      points = elems.split(",")
      coords.append((float(points[0]), float(points[1])))
    coords_names_dict[Polygon(coords)] = name
  return coords_names_dict

# make dataframe from function
import pandas as pd

cpa_polygons = pd.DataFrame(coords_names(placemks_with_polygons).items(),
            columns=['Polygon','CPA'])

In [41]:
pd.DataFrame(Counter(cpa_polygons.CPA).items()).sort([1],ascending=False).head()

,0,1
39,RESERVE,5
20,MILITARY FACILITIES,3
11,OCEAN BEACH,2
0,BLACK MOUNTAIN RANCH,1
42,SAN YSIDRO,1
